**извлечение признаков из текста на естественном языке**

классификатор текстов

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
134,"Демьяненко завтра станет полноправным тренером киевского ""Динамо""В\nчетверг, 8 декабря ожидается назначение Анатолия Демьяненко на пост\nглавного тренера ""Динамо"". Журналистам Демьяненко будет пре...",sport
720,"СК возбудил дело за разжигание национальной розни против двух ""колумнистов-русофилов""\n\nопубликовано: 8 декабря 2016 в 20:44\n\nобновлено: 8 декабря 2016 в 23:33\n\nTUT.BY\n\nСледственный комитет...",politics


## токенизация и очистка

In [6]:
from Stemmer import Stemmer
# pacman -S python-pystemmer
# pip install pystemmer

In [7]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ t for t in tt if t not in ['в','на','за','для','под','над'] ] # удаление (предлогов)
    tt = Stemmer('russian').stemWords( tt ) # стемминг, выделение основы слова
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    tt = [ t for t in tt if len(t)>2 ] # удаление коротких слов (предлогов)
    return ' '.join( [ t.strip() for t in tt if t ] )
    

##  CountVectorizer + TF-IDF

In [8]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(preprocessor=preprocessor)
tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f7fd016a620>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [9]:
# размер словаря
len(tf.vocabulary_)

37661

## формируем датасеты

In [10]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 37661)

In [11]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [12]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

---

In [13]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [14]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9, random_state=get_seed() )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((319, 37661), (319,), (2877, 37661), (2877,))

## обучаем

In [15]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

## тестируем

In [16]:
o = clf.predict(X_train)

In [17]:
accuracy_score(y_train,o)

1.0

In [18]:
# print( classification_report(y_train,o) )

---

In [19]:
o = clf.predict(X_test)

In [20]:
accuracy_score(y_test,o)

0.7869308307264512

In [21]:
print( classification_report(y_test,o) )

              precision    recall  f1-score   support

           0       0.95      0.75      0.84       228
           1       0.73      0.77      0.75       330
           2       0.66      0.88      0.76       241
           3       0.89      0.20      0.32        87
           4       0.76      0.93      0.84       393
           5       0.81      0.92      0.86       536
           6       0.88      0.25      0.39        55
           7       0.57      0.38      0.45        45
           8       0.73      0.98      0.84       203
           9       0.57      0.22      0.32       134
          10       0.91      0.96      0.94       328
          11       0.87      0.66      0.75       263
          12       0.00      0.00      0.00        34

   micro avg       0.79      0.79      0.79      2877
   macro avg       0.72      0.61      0.62      2877
weighted avg       0.78      0.79      0.76      2877



/usr/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


---

In [22]:
from matplotlib import pyplot as plt
import itertools

plt.figure(figsize=(10,9))
cm = confusion_matrix(y_test,o)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()

classes = sorted(labels.keys())

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

<Figure size 1000x900 with 2 Axes>

---

In [23]:
o = clf.predict(X)

In [24]:
labels_inv = { labels[k]:k for k in labels}
# labels_inv

In [25]:
i = np.random.randint(len(data))
print('tag:',data.iloc[i,1])
print('predict:',labels_inv[o[i]])
print('- - - - - - - - - - - - - - - - - - \n')
print(data.iloc[i,0])


tag: tech
predict: tech
- - - - - - - - - - - - - - - - - - 

В отношении медиа-изданий введены некоторые послабления со стороны Роскомнадзора. Речь идёт о том, что агрегаторам позволили не проверять в обязательном порядке публичные данные, размещённые на сайтах масс-медиа.

Существующие положения федерального закона «О новостных агрегаторах», как пояснили специалисты государственного ведомства Роскомнадзор, не будут распространяться в форме мер воздействия по отношению к владельцем новостных агрегаторов. Это станет возможным в случае распространения недостоверной и общественно значимой, а также иной информации, полученный из официальных источников.

Особо отмечено, что не будут проверяться сайты, которые прошли госрегистрацию в качестве СМИ, имеют официальный сайт, разрешённый органами власти. В Роскомнадзоре уточняют, что новостной агрегатор может вполне использовать информацию без дополнительной проверки, но это будет возможно при наличии данных в источниках ее распространения.

Гер

---

In [26]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit(data['tag'])
# print( list(le.classes_) )
# y = le.transform(data['tag']) 

In [27]:
# from sklearn.pipeline import Pipeline
# text_clf = Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('clf', SGDClassifier(loss='hinge')),
#                 ])
# text_clf.fit()

In [ ]:
# from sklearn.model_selection import GridSearchCV

---

In [28]:
# Sebastian Raschka   Python Machine Learning  - Packt Publishing Ltd, 2015